# Introduction
<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to perform Accessory authentication with TFLXTLS device. 

The CryptoAuthentication device supports different modes of operation with MAC command. The mode used here is “Move the Challenge to TempKey” where the host sends a random seed to the device and the device generates a nonce (number used once) which incorporates it. The generated nonce here is the challenge. The application sends the MAC command to the device to generate response with the challenge and the symmetric key in it. The generated response is sent to the application and the application performs the same operation done on the device side to generate the response. Finally, the application compares the response from the device and its own calculated response to authenticate. 

Before running this Notebook,
1. Make sure CryptoAuth Trust Platform is having factory reset program

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange>  azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd()))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TFLXTLS hardware and establishes commmunication with TFLXTLS.
2. Performs device initialization to verify the actual device attached

In [ ]:
import argparse,time
import warnings

from cryptoauthlib import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from asn1crypto import pem
from ipywidgets import widgets

from trustplatform import *

SHARED_SECRET_SLOT = 5 #The slot which contains the symmetric key
MAC_MODE = 0x41

warnings.filterwarnings('ignore')
print("Importing modules - Successful")

#TFLXTLS device I2C address 0x6C; #TNGTLS device I2C address 0x6A; 
common_helper.connect_to_secure_element('ATECC608A', ATCAKitType.ATCA_KIT_I2C_IFACE, 0x6C)
print("Device initialization - Successful")

### Generate MAC from TFLXTLS part
Generate the MAC with ECC608A device.The MAC is generated with the symmetric key in slot, Random number and with device serial number.The MAC generated from the device uses the Random number generator and the generated MAC is unique and will not repeat in future.

<center><img src="img/accessory_auth_image_source1.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
seed_in = bytearray(20)
rand_out = bytearray(32)
nonce = bytearray()
device_mac = bytearray(32)

# Generate the nonce in device and return the random number
assert atcab_nonce_rand(seed_in,rand_out) == Status.ATCA_SUCCESS, "Random nonce from device failed"

# Calculate the nonce value on the host side
nonce.extend(rand_out[0:32])
nonce.extend(seed_in[0:20])
nonce.append(0x16)
nonce.append(0)
nonce.append(0)
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(bytes(nonce))
nonce = digest.finalize()

# Calculate the mac in device with its symmetric diversified key in slot
assert atcab_mac(MAC_MODE, SHARED_SECRET_SLOT, 0, device_mac) == Status.ATCA_SUCCESS, "MAC from Accessory device failed"
print("MAC Received from Accessory device:")
print(common_helper.pretty_print_hex(device_mac, indent='    '))

### Verify Expected MAC on Host
Calculate the MAC on the host side.The host with the help of the symmetric key, random nonce and with the TFLXTLS serial number generates the Expected MAC. It then compares the Expected MAC with the MAC sent from TFLXTLS device and executes application after successfull authentication.

<center><img src="img/accessory_auth_image_source2.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def mac_mac_resp_verify(b):
    serial_number = bytearray(9)
    host_mac = bytearray()
    
    # Symmetric key to be used host mac calculation
    Symmetric_key = bytearray()
    with open(os.path.join(home_path, 'TrustFLEX', '00_resource_generation', 'slot_5_secret_key.pem'), 'rb') as f:
        pem_bytes = f.read()
    type_name, headers, Symmetric_key = pem.unarmor(pem_bytes)
    Symmetric_key = Symmetric_key[17: len(Symmetric_key)]

    #Read the serial number from device
    assert atcab_read_serial_number(serial_number) == Status.ATCA_SUCCESS, "Serial number read from device failed"

    # Calculate the mac on the host side
    host_mac.extend(Symmetric_key[0:32])
    host_mac.extend(nonce[0:32])
    host_mac.append(8)
    host_mac.append(MAC_MODE)
    host_mac.append(SHARED_SECRET_SLOT)
    host_mac.append(0)
    host_mac.extend([0 for i in range(11)])
    host_mac.append(serial_number[8])
    host_mac.extend(serial_number[4:8])
    host_mac.extend(serial_number[0:2])
    host_mac.extend(serial_number[2:4])
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(bytes(host_mac))
    host_mac = digest.finalize()
    
    #uncomment following line to try wrong mac
    #host_mac = bytes(b'00000000000000000000000000000000')

    print('MAC calculated on host:')
    print(common_helper.pretty_print_hex(host_mac, indent='    '))

    if (device_mac == host_mac):
        print('\nAccessory device authenticated successfully!')
        mac_verify.button_style = 'success'
    else:
        mac_verify.button_style = 'danger'
        print('\Accessory device not authenticated...')

tooltip = 'Click to perform MAC-Response Verify'
mac_verify = widgets.Button(description = 'Verify MAC', tooltip=tooltip)
mac_verify.on_click(mac_mac_resp_verify)
display(mac_verify)